In [1]:
# Default parameters
KEY = 'inp'

LANG = 'en-us'
FILE = KEY + '.txt'
FOLDER = KEY
OUT = KEY + '.mp3'
OUT_LINK = KEY + '_link.txt'
CLEAR_FOLDER = False
REPETITION = 3

In [2]:
import  requests
import json
import urllib
from bs4 import BeautifulSoup
import sys
import os
import csv

In [3]:
# for more information on how to install requests
# http://docs.python-requests.org/en/master/user/install/#install

def pronunciation_link(word_id, language = LANG):
    app_id = '4174367d'
    app_key = 'd1bc1cae2ee9be39b8879d216a5dd57e'
    url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()
    url += '?fields=pronunciations&strictMatch=false'
    r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
    try:
        audiolink = r.json()['results'][0]['lexicalEntries'][0]['pronunciations'][1]['audioFile']
        return audiolink
    except:
        print(word_id)
    return ""

In [32]:
# Upload from Merriam Dictionary
# https://dictionaryapi.com/api/v3/references/collegiate/json/test?key=0696dcdd-f909-4959-96ac-dd434d86c832

def pronunciation_link(word_id):
    app_key = '0696dcdd-f909-4959-96ac-dd434d86c832'
    url = 'https://dictionaryapi.com/api/v3/references/collegiate/json/' + word_id.lower()
    url += '?key=' + app_key
    r = requests.get(url)
    try:
        name = r.json()[0]['hwi']['prs'][0]['sound']['audio']
        audiolink = 'https://media.merriam-webster.com/soundc11/' + name[0] + '/' + name + '.wav'
        return audiolink, r.json()
    except:
        print(word_id)
    return ""

In [91]:
def upload_file(url):
    r = requests.get(url, allow_redirects=True)
    return r

In [125]:
r = pronunciation_link('awesome')

In [82]:
res = upload_file(r)
with open('heart.mp3', 'wb') as f:
    f.write(res.content)

# Upload mp3 files to a folder and merge them

In [24]:
# Load once
url = 'https://raw.githubusercontent.com/anars/blank-audio/master/2-seconds-and-500-milliseconds-of-silence.mp3'
f = upload_file(url)
with open('silence.mp3', 'wb') as g:
    g.write(f.content)

In [19]:
# Create a folder
import os
import shutil
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, FOLDER)

if os.path.exists(final_directory):
    if (CLEAR_FOLDER):
        shutil.rmtree(final_directory)
if not os.path.exists(final_directory):
    os.makedirs(final_directory)       

In [39]:
# Read words and upload mp3 files to the folder

with open(FILE) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    for i, row in enumerate(csv_reader):
        word = row[0]
        try:
            url = pronunciation_link(word)
            if (url != ""):
                file = upload_file(url)
                prefix = str(i) if i >= 10 else '0' + str(i)
                file_name = os.path.join(final_directory, prefix + '_' + word + '.mp3')
                with open(file_name, 'wb') as f:
                    f.write(file.content)
        except:
            print(word)

Adjacent
Analysis
Augmentation
AX
Criteria
Process
Resources


In [25]:
silence = open('silence.mp3', 'rb').read()

In [48]:
import os
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, FOLDER)

files = []
res = bytearray()
# r=root, d=directories, f = files
for r, d, f in os.walk(final_directory):
    for file in f:
        if '.mp3' in file:
            file_name = os.path.join(r, file)
            files.append(file_name)


                
#files.sort()
#print(files)
for file_name in files:
    data = open(file_name, 'rb').read()
    res = res + REPETITION * data + silence
    
with open(os.path.join(current_directory, OUT), 'wb') as out:
    out.write(res)

# Upload links to file

In [22]:
# Read words and upload mp3 files to the folder
import csv

f= open(OUT_LINK,"w")

with open(FILE) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    for i, row in enumerate(csv_reader):
        word = row[0].replace(' ', '_')
        link = pronunciation_link(word)
        f.write(";".join(row) + ";" + link + "\n")
f.close()

﻿atavistic
break_into_a_shop
be_accused_of
witnesses_were_called_to_give_evidence
give_evidence
give_a_verdict
sentence_to_5_years
cut_the_music
get_away_with
stay_put
interfer_with_a_police_investigation
break_a_law
get_a_bad_rap_for_sth
meddle_with_sth
cross_the_threshold
pan_out
stick_my_neck_out


# German

In [53]:
# Upload transcription, audio links
def word_links(word_id):
    word = word_id.lower()
    audio = ""
    url = 'https://www.howtopronounce.com/german/'
    url = url + word
    try:
        r = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(r)
        audio = soup.find('span', {'id': 'pronouncedContents'}).find('span').get('onclick').split('\'')
        audio = audio[1]
    except:
        print(word_id)
        print(url)
        print("Oops!",sys.exc_info()[0],"occured.")
    gender = ""
    translation = ""
    eger = ""
    eeng = ""
    url = 'https://www.linguee.com/german-english/translation/'
    url = url + word + '.html'
    try:
        r = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(r)
        g = soup.find('span', {'class': 'tag_wordtype'}).get_text().split(',\xa0')
        if (len(g) >=2):
            if(g[0] == "noun"):
                g = g[1]
                if(g == "feminine"):
                    gender = "die"
                if(g == "neuter"):
                    gender = "das"
                if(g == "masculine"):
                    gender = "der"
        translation = soup.find('span', {'class': 'tag_trans'}).find('a').get_text()
        e = soup.find('span', {'class': 'tag_e'})
        eger = e.find('span', {'class': 'tag_s'}).get_text()
        eeng = e.find('span', {'class': 'tag_t'}).get_text()
    except:
        print(word_id)
        print(url)
        print("Oops!",sys.exc_info()[0],"occured.")
    return audio, gender, translation, eger, eeng

In [54]:
audio, gender, translation, eger, eeng = word_links('vater')

# Enchant word cards with transcription, image, sounds

In [8]:
urls = ['https://dictionary.cambridge.org/us/dictionary/english-russian/', 
        'https://dictionary.cambridge.org/us/dictionary/english/',
        'https://www.dictionary.com/browse/']
transcription_classes = {urls[0]: 'ipa dipa', urls[1]: 'ipa', urls[2]: 'pron-ipa-content'}

In [9]:
def get_audio_0(soup):
    audios = soup.find_all('amp-audio')
    if (len(audios) >= 2):
        audio = audios[1]
    else:
        audio = audios[0]
    audio_link = 'https://dictionary.cambridge.org/' + audio.find('source').get('src')
    return audio_link
def get_audio_1(soup):
    audios = soup.find_all('span', {'class': 'circle circle-btn sound audio_play_button'})
    if (len(audios) >= 2):
        audio = audios[1]
    else:
        audio = audios[0]
    audio_link = 'https://dictionary.cambridge.org/' + audio.get('data-src-mp3')
    return audio_link
def get_audio_2(soup):
    return soup.find('span', {'class': 'audio-wrapper'}).find('source', {'type': 'audio/mpeg'}).get('src')

In [10]:
get_audio_methods = {urls[0]: get_audio_0, urls[1]: get_audio_1, urls[2]: get_audio_2}

In [11]:
# Upload transcription, audio links
def word_links(word_id, transcription_classes = {}, get_audio_methods = {}):
    transcription = ""
    audio = ""
    for i in range(len(urls)):
        url = urls[i] + word_id.lower()
        try:
            r = urllib.request.urlopen(url).read()
            soup = BeautifulSoup(r)
            if (len(transcription) == 0 and len(transcription_classes) > 0):
                transcription = soup.find('span', {'class': transcription_classes[urls[i]]}).get_text()
            if (len(audio) == 0 and len(get_audio_methods) > 0):
                audio = get_audio_methods[urls[i]](soup)
            if (not (len(transcription) == 0 and len(transcription_classes) > 0) and 
                not(len(audio) == 0 and len(get_audio_methods) > 0)):
                break
        except:
            print(word_id)
            print(url)
            print("Oops!",sys.exc_info()[0],"occured.")
    return transcription, audio

In [12]:
word_links('mother', get_audio_methods = get_audio_methods)

('',
 'https://dictionary.cambridge.org//us/media/english-russian/us_pron/m/mot/mothe/mother.mp3')

In [15]:
# Read words and upload mp3 links
import csv

f= open(OUT_LINK,"w")

with open(FILE) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    for i, row in enumerate(csv_reader):
        if not(" " in row[0]):
            word = row[0]
            #print(word)
            #links = word_links(word, transcription_classes = transcription_classes, get_audio_methods = get_audio_methods)
            #links = word_links(word, transcription_classes = transcription_classes)
            links = word_links(word, get_audio_methods = get_audio_methods)
            f.write(";".join(row) + ";" + ";".join(links) + "\n")
f.close()

uteruse
https://dictionary.cambridge.org/us/dictionary/english-russian/uteruse
Oops! <class 'IndexError'> occured.
uteruse
https://dictionary.cambridge.org/us/dictionary/english/uteruse
Oops! <class 'IndexError'> occured.
uteruse
https://www.dictionary.com/browse/uteruse
Oops! <class 'urllib.error.HTTPError'> occured.
fetuse
https://dictionary.cambridge.org/us/dictionary/english-russian/fetuse
Oops! <class 'IndexError'> occured.
fetuse
https://dictionary.cambridge.org/us/dictionary/english/fetuse
Oops! <class 'IndexError'> occured.
fetuse
https://www.dictionary.com/browse/fetuse
Oops! <class 'urllib.error.HTTPError'> occured.
rivarly
https://dictionary.cambridge.org/us/dictionary/english-russian/rivarly
Oops! <class 'IndexError'> occured.
rivarly
https://dictionary.cambridge.org/us/dictionary/english/rivarly
Oops! <class 'IndexError'> occured.
rivarly
https://www.dictionary.com/browse/rivarly
Oops! <class 'urllib.error.HTTPError'> occured.


In [44]:
# Upload link to image for word
# https://unsplash.com/developers
def image_link(word_id):    
    access_key = 'bd390348ffe9b3098bd4352d244370cde79d962026d8e6bb33e7b0aaae7d7a74'
    secret_key = 'db9fb822747a679d347cb815f08c299c614eaad386784b39540cd763b761ab3c'
    url = 'https://api.unsplash.com/search/photos?client_id=' + access_key + '&client_secret=' + secret_key +'&query=' + word_id.lower()
    r = requests.get(url)
    try:
        name = r.json()
        image_link = name['results'][0]['urls']['regular']
        return image_link
    except:
        print(word_id, "Image")
        print("Oops!",sys.exc_info()[0],"occured.")
    return ""

In [ ]:
# Small test
url = image_link('persistent issue')
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= url)

In [6]:
# Read words and upload mp3 files to the folder
import csv

f= open(OUT_LINK,"w")

with open(FILE) as csv_file:
    csv_reader = csv.reader(csv_file)
    for i, row in enumerate(csv_reader):
        words = row[0].split(' ')
        print(words)
        for word in words:
            links = word_links(word)
            image = image_link(word)
            f.write(word + "@" + "@".join(links) + "@" + image + "@")
            
        f.write('\n')
        
f.close()

['balm', 'bum']
['bastard', 'bustard']
['branch', 'brunch']
['calf', 'cuff']
['calm', 'come']
["can't", 'cunt']
cunt Image
Oops! <class 'IndexError'> occured.
['dance', 'dunce']
dunce Image
Oops! <class 'IndexError'> occured.
['dramas', 'drummers']
['fast', 'fussed']
fussed Image
Oops! <class 'IndexError'> occured.
['grant', 'grunt']
grunt Image
Oops! <class 'IndexError'> occured.
['graph', 'gruff']
['haft', 'huffed']
haft Image
Oops! <class 'IndexError'> occured.
huffed Image
Oops! <class 'IndexError'> occured.
['half', 'huff']
['last', 'lust']
['staff', 'stuff']
['task', 'tusk']
['cart', 'cut']
['carb', 'vub']
vub Dictionary
Oops! <class 'urllib.error.HTTPError'> occured.
vub Image
Oops! <class 'IndexError'> occured.
['barn', 'bun']
['charm', 'chum']
['garner', 'gunner']
['heart', 'hut']
['March', 'much']
['dark', 'duck']


In [3]:
# Upload link to image for word
# https://unsplash.com/developers
def image_links(word_id):
   
    access_key = 'bd390348ffe9b3098bd4352d244370cde79d962026d8e6bb33e7b0aaae7d7a74'
    secret_key = 'db9fb822747a679d347cb815f08c299c614eaad386784b39540cd763b761ab3c'
    url = 'https://api.unsplash.com/search/photos?client_id=' + access_key + '&client_secret=' + secret_key +'&query=' + word_id.lower()
    r = requests.get(url)
    try:
        name = r.json()
        return name['results']
        #image_link = name['results'][0]['urls']['regular']
    except:
        print(word_id, "Image")
        print("Oops!",sys.exc_info()[0],"occured.")
    return ""

In [5]:
from IPython.display import Image, display
from IPython.core.display import HTML 
IMGCOUNT = 5

with open(FILE) as f:
    lines = f.read().split('\n')
    for word in lines:
        print(word)
        links = image_links(word)
        count = min(len(links), IMGCOUNT)
        for i in range(count):
            link = links[i]['urls']['regular']
            print(link)
            display(Image(url = link, width = 200))

attentiveness
https://images.unsplash.com/photo-1531986907323-659a9efa41b9?ixlib=rb-1.2.1&q=80&fm=jpg&crop=entropy&cs=tinysrgb&w=1080&fit=max&ixid=eyJhcHBfaWQiOjk4NTU4fQ


administer something to somebody
https://images.unsplash.com/photo-1484663548870-2aa675ba38fb?ixlib=rb-1.2.1&q=80&fm=jpg&crop=entropy&cs=tinysrgb&w=1080&fit=max&ixid=eyJhcHBfaWQiOjk4NTU4fQ


https://images.unsplash.com/photo-1561799113-3eb2e87c626f?ixlib=rb-1.2.1&q=80&fm=jpg&crop=entropy&cs=tinysrgb&w=1080&fit=max&ixid=eyJhcHBfaWQiOjk4NTU4fQ


https://images.unsplash.com/photo-1537549603790-54c64b32b848?ixlib=rb-1.2.1&q=80&fm=jpg&crop=entropy&cs=tinysrgb&w=1080&fit=max&ixid=eyJhcHBfaWQiOjk4NTU4fQ


https://images.unsplash.com/photo-1484071096222-7936a931e094?ixlib=rb-1.2.1&q=80&fm=jpg&crop=entropy&cs=tinysrgb&w=1080&fit=max&ixid=eyJhcHBfaWQiOjk4NTU4fQ


...

the findings reveal that
https://images.unsplash.com/photo-1486148727607-f97aa04df01e?ixlib=rb-1.2.1&q=80&fm=jpg&crop=entropy&cs=tinysrgb&w=1080&fit=max&ixid=eyJhcHBfaWQiOjk4NTU4fQ


https://images.unsplash.com/photo-1505403858922-407e2bfdf959?ixlib=rb-1.2.1&q=80&fm=jpg&crop=entropy&cs=tinysrgb&w=1080&fit=max&ixid=eyJhcHBfaWQiOjk4NTU4fQ


https://images.unsplash.com/photo-1484820986637-7ec3e85b394f?ixlib=rb-1.2.1&q=80&fm=jpg&crop=entropy&cs=tinysrgb&w=1080&fit=max&ixid=eyJhcHBfaWQiOjk4NTU4fQ


https://images.unsplash.com/photo-1522024343969-805a5eadd461?ixlib=rb-1.2.1&q=80&fm=jpg&crop=entropy&cs=tinysrgb&w=1080&fit=max&ixid=eyJhcHBfaWQiOjk4NTU4fQ


https://images.unsplash.com/photo-1552842027-7a06d816bb2a?ixlib=rb-1.2.1&q=80&fm=jpg&crop=entropy&cs=tinysrgb&w=1080&fit=max&ixid=eyJhcHBfaWQiOjk4NTU4fQ


# Download images from Quizlet

In [3]:
import pprint as pp
def image_links(file_name):
    file = open(file_name, "r")
    f= open(file_name.split('.')[0] + '_out.txt',"w")
    r = file.read()
    soup = BeautifulSoup(r)
    try:
        words = soup.find_all('a', {'class': 'SetPageTerm-image'})
        print(words)
        links = [e['style'].split('"')[1] for e in words]
        for link in links:
            f.write(link + '\n')
        pp.pprint(links)
    except:
        pass
    f.close()
    return ""

In [4]:
image_links('tmp.html')

[<a class="SetPageTerm-image" style='background-image: url("https://farm5.staticflickr.com/4013/4495767579_c8ba84b1f7.jpg");'></a>, <a class="SetPageTerm-image" style='background-image: url("https://o.quizlet.com/i/uciWMuqwyE5ZOOeEovsbGg.jpg");'></a>, <a class="SetPageTerm-image" style='background-image: url("https://o.quizlet.com/OBGqYbuNWZXoJwZ71cfF5w.jpg");'></a>, <a class="SetPageTerm-image" style='background-image: url("https://o.quizlet.com/i/T7i2NA_MrZ-NnUEQ9nNOtQ.jpg");'></a>, <a class="SetPageTerm-image" style='background-image: url("https://o.quizlet.com/ttJiTZoMBHee-VsrQIlD1A.jpg");'></a>, <a class="SetPageTerm-image" style='background-image: url("https://o.quizlet.com/cV8xBgsVO0XAT9JbcEwtvg.jpg");'></a>, <a class="SetPageTerm-image" style='background-image: url("https://o.quizlet.com/f1RzC62d-kj4aqhu1SOFSg.jpg");'></a>, <a class="SetPageTerm-image" style='background-image: url("https://farm1.staticflickr.com/96/281886153_0b29297256.jpg");'></a>, <a class="SetPageTerm-image"

''

# Download sounds from LinguaLeo

In [34]:
import pprint as pp
def sound_links(file_name):
    file = open(file_name, "r")
    f= open(file_name.split('.')[0] + '_out.txt',"w")
    r = file.read()
    soup = BeautifulSoup(r)
    try:
        sounds = soup.find_all('div', {'class': 'sets-words__row false'})
        for sound in sounds:
            audio = sound.find('audio')['src']
            word = sound.find('strong', {'class': 'sets-words__my-word'}).get_text()
            f.write(word + ";" + audio + ";\n")
    except:
        pass
    return ""

In [35]:
sound_links('tmp.html')

''

# Check Anki images

In [12]:
from IPython.display import Image, display
from IPython.core.display import HTML 


with open(FILE) as f:
    lines = f.read().split('\n')
    for line in lines:
        strs = line.split('\t')
        #print(strs)
        name = strs[0]
        image = strs[2]
        print(name)
        display(Image(url = image))

peppermint


tripe


bogey


incantation


rest on your laurels


...

embezzlement


forgery


interrogate


rape


insolence


# Rename mp3 files

In [12]:
names_file = "D:\\pron\\names.txt"
names = []
with open(names_file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ')
    for i, row in enumerate(csv_reader):
        names.append(row[0])
        names.append(row[1])
folder = "D:\pron"
import os
i = 0
#j = 0
for file in os.listdir(folder):
    if file.endswith(".mp3"):
        #if j % 2 == 0:
        old = os.path.join(folder, file)
        new = old[:-5] + names[i] + '.mp3'
        i = i + 1
        #print(old)
        #print(new)
        os.rename(old, new)
        #j += 1